# Python実行環境のテスト

　この Notebook は、農研機構メッシュ農業気象データシステムの利用設定が正しく行われたかを確認するためのものです。
 以下、4つのCode Cell を順次実行すると、茨城県つくば市館野における最近の気温データを取得し、その推移を折れ線グラフにして表示するとともに、それを画像のPNGファイルとして作成します。

### 1 ライブラリーのインポート
メッシュ農業気象データを利用するのに必要なライブラリをインポートします。気象データの折れ線作成には使用しないライブラリについても、インストール確認のためインポートしています。

　エラーが表示されなければOKです。
 * 図で日本語を使用したい場合は、**AMD_Tools4.pyの89-90行目**のコメントアウトを外して有効にしてください。その際、コメントをよく読みOSの違いにご注意ください。

In [ ]:
# モジュールのインポート
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as md
import AMD_Tools4 as amd
import xarray as xr
import pandas as pd

### 2 取得する気象データの時空間領域の設定
　AMGSDSにリクエストする気象データの気象要素、空間領域、期間を設定します。プログラムを実行した日の前後64日間の日付をプログラムで自動生成するので若干複雑になっています。

 エラーが表示されなければOKです。

In [ ]:
# 計算の領域と期間の指定
nani = "TMP_mea"
doko = [ 36.0566, 36.0566, 140.125, 140.125]	#つくば(舘野)
p = datetime.today()				#今日の時刻オブジェクト
db = p - timedelta(days=64)		#期間の始まり、今日-days
de = p + timedelta(days=64)		#期間の終わり、今日+days
date_begin = db.strftime("%Y-%m-%d")
date_end   = de.strftime("%Y-%m-%d")
itsu = [ date_begin, date_end ]

### 3 気象データの読み込み
ライブラリ AMD_Tools3 が提供する関数 **GetMetData** の引数にデータ取得範囲他を与えて、AMGDSDのデータ配信サーバから所定の気象データを取得します。気象値と平年値を取得するので関数は2回使われています。

　実行後、少し時間をおいて、以下が表示されればOKです。
```
Cli_TMP_mea (129, 1, 1)
TMP_mea (129, 1, 1)
```


In [ ]:
# メッシュデータの取得
T0, tim, lat, lon = amd.GetMetData(nani, itsu, doko, cli=True)
T1, tim, lat, lon, nam, uni = amd.GetMetData(nani, itsu, doko, namuni=True)
T0 = T0[:,0,0]
T1 = T1[:,0,0]

### 4 折れ線グラフの描画と画像ファイル出力
　ライブラリ matplotlib の機能を用いて、取得した気象データの折れ線グラフを作成するとともに、表示した画像をファイルにして出力します。 matplotlib は美しいグラフを作成することができますが、その分、設定項目が多く初心者には難解です。初めての方は理解できなくて構いません。
 
　グラフが表示され、ファイル **result.png** が生成されていればOKです。

In [ ]:
# 時系列グラフの描画
D1D1 = T1
D1D2 = T0
title = "N"+str(doko[0])+", E"+str(doko[2])+' ('+p.strftime("%Y/%m/%d")+')'
fig = plt.figure(num=None, figsize=(12, 4))
# ・目盛の作成
ax = plt.axes()
xmajoPos = md.DayLocator(bymonthday=[1])
xmajoFmt = md.DateFormatter('%m/%d')
ax.xaxis.set_major_locator(xmajoPos)
ax.xaxis.set_major_formatter(xmajoFmt)
xminoPos = md.DayLocator()
ax.xaxis.set_minor_locator(xminoPos)
# ・データのプロット
ax.fill_between(tim,D1D1,D1D2,where=D1D1>D1D2,facecolor='orange',alpha=0.5,interpolate=True)	#細線より高い部分を橙色に塗る
ax.fill_between(tim,D1D2,D1D1,where=D1D1<D1D2,facecolor='skyblue',alpha=0.5,interpolate=True)	#細線より低い部分を水色に塗る
ax.plot(tim, D1D1, 'k') #太線
ax.plot(tim, D1D2, 'k', linewidth=0.3) #細線
# ・「今日」印を付ける
p = datetime.today() #今日の時刻オブジェクト
today = tim == datetime(p.year,p.month,p.day,0,0,0) #今日の配列要素番号
plt.plot(tim[today], D1D1[today], "ro") #今日に赤点を打つ
# ・ラベル、タイトルの付加
plt.xlabel('Date')
plt.ylabel(nam+' [' + uni + ']')
plt.title(title)
plt.savefig('result'+'.png', dpi=600) #この文をコメントアウトすると、図のpngファイルは作られません。
plt.show()
